<center>                                       <H1>Validation of the NN code generator for the 2D diffusion equation </H1> 
    <H3>O. Pannekoucke</H3> 
</center>
$   %\newcommand{\pde}{\partial}
    %\newcommand{\pdt}{\partial_t}
    %\newcommand{\pdx}{\partial_x}
    \newcommand{\bu}{\bar u}
    \newcommand{\eps}{\varepsilon}$

<center> <b>Objectifs</b> </center>

 * Definition of the 2D diffusion equation by using `sympy`
 * Computation of the numerical solution of the 2D diffusion using a NN

--- 
<h1><center>Contents</center></h1>


  1. [Introduction](#intro)
  1. [Dynamics](#model)
  1. [Numerical code for the resolution](#code)
  1. [Numerical application](#num)
  1. [Conclusion](#conclusion)
---

## Introduction <a id='intro'/>

The aim is to compute the solution of the diffusion equation given by
$$\pdt u = \partial_{x^i}\left(\kappa_{ij}\partial_{x^j} u \right),$$
where $\kappa$  is a field of diffusion tensors.

## Dynamics <a id='model'>

In [ ]:
from sympy import Function, symbols, init_printing, Derivative, Matrix
from pdenetgen import Eq, CoordinateSystem
init_printing() 

**Set the diffusion equation**

In [ ]:
t, x, y = symbols('t x y')
u = Function('u')(t,x,y)
kappa11 = Function('\\kappa_{11}')(x,y)
kappa12 = Function('\\kappa_{12}')(x,y)
kappa22 = Function('\\kappa_{22}')(x,y)

diffusion_in_2D = Eq(Derivative(u,t),
  Derivative(kappa11*Derivative(u,x)+kappa12*Derivative(u,y) ,x)+
  Derivative(kappa12*Derivative(u,x)+kappa22*Derivative(u,y) ,y)).doit()

## Numerical code for the resolution<a id='code'>

### Symbolic to numerical code

In [ ]:
from pdenetgen import NNModelBuilder

In [ ]:
cas_model = NNModelBuilder(diffusion_in_2D, class_name="NNDiffusion2DHeterogenous")

In [ ]:
cas_model.constant_functions

In [ ]:
print(cas_model.code)

In [ ]:
infile = False
if infile:
    # -1- Write module
    cas_model.write_module()
    # -2- Load module
    exec(f"from {cas_model.module_name} import {cas_model.class_name}")
else:
    exec(cas_model.code)

## Numerical application <a id='num'/>

### Definition of the domain of computation

The domain is the bi-periodic sqare $[0,1)\times [0,1)$

### Set the numerical NN model

In [ ]:
num_model = NNDiffusion2DHeterogenous()
domain = num_model

**Set initial fields**

In [ ]:
import numpy as np

In [ ]:
dx, dy = num_model.dx

# Set a dirac at the center of the domain.
U = np.zeros(num_model.shape)
U[num_model.shape[0]//2, num_model.shape[0]//2] = 1./(dx*dy)



In [ ]:
num_model.shape

In [ ]:
X = np.asarray(num_model.X)
k = np.asarray([1,2])
X = np.moveaxis(X,0,2)
print(X.shape)

np.linalg.norm(X@k -k[0]*num_model.X[0]-k[1]*num_model.X[1])

**Set constants and time step**

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
time_scale = 1.
#
# Construction du tenseur de diffusion
#

# a) Définition des composantes principales
lx, ly = 10*dx, 5*dy
kappa_11 = lx**2/time_scale
kappa_22 = ly**2/time_scale

# b) Construction d'un matrice de rotation
R = lambda theta : np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])

# c) Spectre du tenseur de référence
D = np.diag([kappa_11,kappa_22])


# d) Set diffusion tensors field
num_model.kappa_11 = np.zeros(num_model.shape)
num_model.kappa_12 = np.zeros(num_model.shape)
num_model.kappa_22 = np.zeros(num_model.shape)

X = np.moveaxis(np.asarray(num_model.X),0,2)
k = 2*np.pi*np.array([2,3])

theta = np.pi/3*np.cos(X@k)
#plt.contourf(*num_model.x, theta)

for i in range(num_model.shape[0]):
    for j in range(num_model.shape[1]):
        lR = R(theta[i,j])
        nu = lR@np.diag([kappa_11,kappa_22])@lR.T
        num_model.kappa_11[i,j] = nu[0,0]
        num_model.kappa_12[i,j] = nu[0,1]
        num_model.kappa_22[i,j] = nu[1,1]
        
        
num_model.kappa_11 = num_model.kappa_11.reshape((1,100,100,1))
num_model.kappa_12 = num_model.kappa_12.reshape((1,100,100,1))
num_model.kappa_22 = num_model.kappa_22.reshape((1,100,100,1))
#
# Calcul du pas de temps adapté au problème
#
dt = np.min([dx**2/kappa_11, dy**2/kappa_22])

CFL = 1/6
num_model._dt = CFL * dt
print('time step:', num_model._dt)

**Illustrates trend at initial condition**

In [ ]:
def plot(field):
    plt.contourf(*num_model.x, field.T)

In [ ]:
state0 = U.copy().reshape((1,1)+num_model.shape+(1,))
print(state0.shape)

In [ ]:
import keras

In [ ]:
dU, = num_model.trend(0,state0)

plot(dU[0].reshape((100,100)))
plt.title('Trend for the diffusion')

**Short forecast**

In [ ]:
times = num_model.window(time_scale)
#saved_times = times[::100]
saved_times = times

In [ ]:
num_model.set_time_scheme('euler')
traj = num_model.forecast(times, state0, saved_times)

In [ ]:
plt.figure(figsize=(12,5))

start, end = [traj[time] for time in [saved_times[0], saved_times[-1]]]

title = ['start', 'end']
for k, state in enumerate([start, end]):
    plt.subplot(121+k)
    plot(state[0].reshape((100,100)))
    plt.title(title[k])
    
plt.savefig('./figures/NN-diffusion-2D-prediction.pdf')
np.save('nn-diffusion.npy', end)

**Comparison with the finite difference solution**

In [ ]:
fd_end_solution = np.load('fd-diffusion.npy')

In [ ]:
np.linalg.norm(fd_end_solution[0] - end[0,0,:,:,0])

## Conclusion <a id='conclusion'/>

In this notebook, the solution of the diffusion equation using a NN code has been presented. 
The results are those of the finite-difference solution (not shown here). This validate the NN generator and illustrates how the physical equations can be used for the design of a NN architecture.